In [1]:
#| default_exp create_embeddings

%load_ext autoreload
%autoreload 2

In [2]:
#| export

from typing import List, Optional
import torch
from transformers import PreTrainedModel, PreTrainedTokenizer, BatchEncoding
from transformers import AutoTokenizer, AutoModel
from sentence_transformers.models import Pooling
import sys
import os
import json
import numpy as np
import time
from tqdm.auto import tqdm
import torch.nn.functional as F

from dotenv import load_dotenv

load_dotenv()

sys.path.append("..")  # Adds the parent directory to sys path

from mailio_ai_libs.chunking import Chunker
from data_types.email import Email, MessageType

/Users/igor/workspace/mailio-ai/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model_id = os.getenv("MODEL_ID")
print("model id: ", model_id)

model id:  intfloat/e5-small-v2


In [4]:
#| export

class Embedder:

    def __init__(self, model: PreTrainedModel, tokenizer: PreTrainedTokenizer):
        self.model = model
        self.model.eval()
        self.tokenizer = tokenizer
        self.max_length = self.model.config.max_position_embeddings  # Model-specific max lengt
        self.chunker = Chunker(tokenizer, chunk_size=self.max_length-2, chunk_overlap=0)
    
    def embed(self, text:List[str]) -> torch.Tensor:
        """
        Generate embeddings for a list of texts.
        
        Args:
            text (List[str]): List of input texts.
            chunk_size (Optional[int]): If text is too long, split it into chunks of this size. If None, no chunking is done.
        
        Returns:
            numpy.ndarray: Embeddings for the input texts.
        """
        
        # Tokenize the input text
        chunks = self.chunker.chunk(".".join(text))
        embeddings = []

        input_texts = []
        for ch in chunks:
            input_texts.append(ch)
        
        inputs = self.tokenizer(input_texts, padding=True, truncation=True, return_tensors="pt")
        inputs = inputs.to(self.model.device)

        with torch.no_grad():
            # Forward pass
            outputs = self.model(**inputs)
        
        # Perform pooling.
        sentence_embeddings = self.mean_pooling(outputs, inputs['attention_mask'])

        # Convert to numpy array and return
        # Move tensors in the list to CPU and convert them to numpy arrays
        return sentence_embeddings.cpu().numpy()

        #Mean Pooling - Take attention mask into account for correct averaging
    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)



In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id)

embedder = Embedder(model, tokenizer)

print(f"loaded model: ", model.config)

loaded model:  BertConfig {
  "_attn_implementation_autoset": true,
  "_name_or_path": "intfloat/e5-small-v2",
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.48.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



In [7]:
a1 = embedder.embed(["That is a happy dog"])
a2 = embedder.embed(["That is a very happy person"])
a3 = embedder.embed(["Today is a sunny day"])

embeddings = np.vstack([a1, a2, a3])
emb = torch.from_numpy(embeddings)
query = torch.from_numpy(embedder.embed(["That is a happy person"]))
print(emb.shape, query.shape)

# Calculate cosine similarity
cosine_scores = F.cosine_similarity(query, emb, dim=1)
print(cosine_scores)

torch.Size([3, 384]) torch.Size([1, 384])
tensor([0.9309, 0.9633, 0.8184])


In [8]:
# list all jsonl files from data (inbox, archive, goodreads,...)
data_dir = '../data'
files = [f for f in os.listdir(data_dir) if f.endswith(".jsonl") and os.path.isfile(os.path.join(data_dir, f))]

output_subfolder = model_id.split("/")[-1]
output_folder = f"../data/{output_subfolder}"
os.makedirs(output_folder, exist_ok=True)

emebddings_filename = "embeddings"

all_embeddings_index = []
all_embeddings = []
all_embeddings_timing = []

# create embeddings
for file in files:

    with open(os.path.join(data_dir, file), 'r') as f:
        jsonl = f.read()
    data = jsonl.split('\n')
    data = [json.loads(d) for d in data if d]
    for item in tqdm(data, desc="Processing emails for {}".format(file)):
        email = Email.from_dict(item)
        text = []
        if email.sender_name:
            text.append(f"sent from {email.sender_name}")
        if email.subject:
            text = [email.subject]
        if len(email.sentences) > 0:
            text.extend(email.sentences)
        
        if len(text) == 0: # nothing to embed
            continue
        
        start_time = time.time()
        embeddings = embedder.embed(text)
        elapsed_time = time.time() - start_time
        all_embeddings_timing.append(elapsed_time)

        all_embeddings.extend(embeddings)
        # add to index number of == len(embeddings) 
        all_embeddings_index.extend([email.message_id] * len(embeddings))

        assert len(all_embeddings_index) == len(all_embeddings)

Processing emails for emails_archive.jsonl: 100%|██████████| 5714/5714 [03:06<00:00, 30.56it/s]


In [9]:
len(all_embeddings), len(all_embeddings_index), len(all_embeddings_timing)

(16328, 16328, 12432)

In [10]:
np_emebddings = np.vstack(all_embeddings)
embeddings_index = np.array(all_embeddings_index)

print(np_emebddings.shape, embeddings_index.shape)

(16328, 384) (16328,)


In [11]:
# Save the embeddings to a file
with open(os.path.join(output_folder, f"{emebddings_filename}.npy"), 'wb') as f:
    np.save(f, np_emebddings)

with open(os.path.join(output_folder, f"{emebddings_filename}_index.npy"), 'wb') as f:
    np.save(f, embeddings_index)

In [12]:
average_time = sum(all_embeddings_timing) / len(all_embeddings_timing)
print(f"\nAverage embedding time: {average_time:.4f} seconds")


Average embedding time: 0.0385 seconds


In [13]:
#| hide

import nbdev; nbdev.nbdev_export()